# Coronavirus(Covid-19) Data Analysis| World & India

https://en.wikipedia.org/wiki/Coronavirus_disease_2019

Date: 19-Mar-2020  (First version)

Author: Karthik C Sunil

Outbroke in Nov 2019 in Wuhan, Mainland China. Spread across the world. 

As I am writing the desease is still spreding in India as well as entire world. 

<<<< STAY SAFE >>>>

In [ ]:
!pip install chart_studio

In [ ]:
# Importing packages

import numpy as np
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
import chart_studio
from plotly.subplots import make_subplots
chart_studio.tools.set_credentials_file(username='karthikcs', api_key='wP88IYO98oHtM1GWuofj')

Load the data `covid_19_data.csv`  from source below:

Original  source is from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

The Data is refreshed to this Kernel Data on a daily basis by the Data owner

# Latest Data Update on

In [ ]:
# Load data
df_country_summary = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')
print('Shape of the data: ', df_country_summary.shape)
print ('Last update: ' + str(df_country_summary.ObservationDate.max()))
df_country_summary = df_country_summary.rename(columns={"ObservationDate": "Date", "Country/Region": "Country"})
# df_country_summary.head()

In [ ]:
def add_rates(df):
    df['Confirmed Change'] = df['Confirmed'].diff()
 
    df['Mortality Rate'] = df['Deaths'] / (df['Confirmed'] - df['Confirmed Change'])
    df['Recovery Rate'] = df['Recovered'] / (df['Confirmed'] - df['Confirmed Change'])
    df['Growth Rate'] = df['Confirmed Change'] / (df['Confirmed'] - df['Confirmed Change'])
    df['Growth Rate Change'] = df['Growth Rate'].diff()
    df['Growth Rate Accel'] = df['Growth Rate Change'] / (df['Growth Rate'] - df['Growth Rate Change'])
    return df

# Analysis of the cases around the World

In [ ]:
world_df = df_country_summary.groupby(['Date']).agg({'Confirmed': ['sum'], 'Recovered': ['sum'], 'Deaths': ['sum']}).reset_index()
world_df.columns = world_df.columns.get_level_values(0)
world_df = add_rates(world_df)
# world_df.head()

In [ ]:
def plot_aggregate_metrics(df, fig=None):
    if fig is None:
        fig = go.Figure()
    fig.update_layout(template='plotly_dark')
#     fig.update_yaxes(type="log")
    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Confirmed'],
                             mode='lines+markers',
                             name='Confirmed',
                             line=dict(color='Yellow', width=2)
                            ))
    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Deaths'],
                             mode='lines+markers',
                             name='Deaths',
                             line=dict(color='Red', width=2)
                            ))
    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Recovered'],
                             mode='lines+markers',
                             name='Recovered',
                             line=dict(color='Green', width=2)
                            ))

    return fig

In [ ]:
fig = plot_aggregate_metrics(world_df)
fig.update_layout(title = 'No. of Covid-19 cases| Growth since 22-Jan-2020 | Across the world')
fig.show()
py.plot(fig, filename='corona-worldwide')

We can see that, Confirmed cases are growing exponentially with a daily increase of around 10 - 15% worldwide <br>
We can also notice that recoveries are also happening at around 25% (I am writing on 26-Mar-2020). But it may vary in later time<br>
But what gives some relaxation is that mortality rate is not that bad. Around 5%

In [ ]:
# Worldwide Rates
def plot_diff_metrics(df, fig=None):
    if fig is None:
        fig = go.Figure()

    fig.update_layout(template='plotly_dark')
    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Mortality Rate'],
                             mode='lines+markers',
                             name='Mortality rate',
                             line=dict(color='red', width=2)))

    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Recovery Rate'],
                             mode='lines+markers',
                             name='Recovery rate',
                             line=dict(color='Green', width=2)))

    fig.add_trace(go.Scatter(x=df['Date'], 
                             y=df['Growth Rate'],
                             mode='lines+markers',
                             name='Growth rate confirmed',
                             line=dict(color='Yellow', width=2)))
    fig.update_layout(yaxis=dict(tickformat=".2%"))
    
    return fig

Let us now see the trends in these rates - New Confirmed Cases, Recovery rate as well as Mortality rate

In [ ]:
fig = plot_diff_metrics(world_df)
fig.update_layout(title = 'World Metrics of Covid-19 Cases: Mortality Rate, Recovery Rate and New Cases rate')
fig.show()

In [ ]:
# fig = go.Figure()

tmp_df = world_df.copy()
tmp_df = tmp_df[tmp_df['Growth Rate Accel'] < 10]

fig = px.bar(tmp_df, x='Date', y='Growth Rate Accel')
fig.update_layout(title = 'Daily Growth Rate Acceleration Worldwide', template='plotly_dark')
fig.update_yaxes(title_text='% Growth Rate Acceleration')
fig.update_layout(yaxis=dict(tickformat=".2%"))
fig.show()

# Cases by Country

To make the visualization better, we will see only top countries and countries similar to India only

In [ ]:
confirmed_by_country_df = df_country_summary.groupby(['Date', 'Country']).sum().reset_index()
by_country_df = confirmed_by_country_df.sort_values(['Country', 'Date'], ascending=[1,1])

In [ ]:
day_ctr = 0
curr_country = None 
country_100plus = []
for ind in by_country_df.index:    
    i_confirmed = by_country_df['Confirmed'][ind]
    i_country   = by_country_df['Country'][ind]
    i_date      = by_country_df['Date'][ind]
    if i_confirmed < 100:
        continue    
    if curr_country == None:
        curr_country = i_country
    elif curr_country != i_country:
        # New country is found reset the day counter 
        day_ctr = 0
        curr_country = i_country
    else:
        day_ctr += 1
        
    country_100plus.append([i_country, i_confirmed, day_ctr])
    
df_country_100plus = pd.DataFrame(country_100plus, columns =['Country', 'Confirmed', 'Days'])

In [ ]:
#Create dataframes for each country for more analysis
# Try to get unique list of countries in the df
countries = by_country_df.Country.unique()
country_df = []
for country in countries:
    country_data = by_country_df[by_country_df['Country'] == country]
    country_data = add_rates(country_data)
    country_data['Days'] = 0
    day_ctr = -1
    for ind in country_data.index:
        i_confirmed = by_country_df['Confirmed'][ind]
        if i_confirmed < 100:
#             country_data['Days'][ind] = day_ctr
            country_data.loc[ind,'Days'] = day_ctr
        else:
            day_ctr += 1
            country_data.loc[ind,'Days'] = day_ctr
    
    
    country_df.append([country,country_data])
    
country_df = pd.DataFrame(country_df, columns = ['Country','Cases'])
country_df = country_df.set_index(['Country'])

In [ ]:
# df_population = pd.read_html('https://www.worldometers.info/world-population/population-by-country/')
df_population = pd.read_csv('/kaggle/input/population-by-country-2020/population_by_country_2020.csv')
# df_population = df_population[0]
df_population = df_population.rename(columns={"Country (or dependency)": "Country", "Population (2020)": "Population"})
# df_population.iloc[0]['Country'] = 'Mainland China'
df_population = df_population.replace("China", "Mainland China")
df_population = df_population.replace("United States", "US")  
df_population = df_population.replace("United Kingdom", "UK")  
df_population = df_population.rename(columns={"Density (P/Km²)": "Density"} )
# df_population.sort_values(['Density (P/Km²)'], ascending=False).head(10)
df_population.head()

In [ ]:
def get_latest_confirmed(country):
    df = country_df.loc[country]['Cases']
    latest_date = df['Date'].max()
    return df[df['Date'] == latest_date]['Confirmed'].iloc[0]

def get_population(country):
    try :
        population = df_population.query("Country == @country ")['Population'].iloc[0]

    except :
        population = -1
    return population

def get_density(country):
    try :
        density = df_population.query("Country == @country ")['Density'].iloc[0]

    except :
        density = -1
    return density

In [ ]:
# df_population[df_population['Country'] == 'Iceland']['Population'].iloc[0]\
df = country_df.reset_index()
df['Latest Confirmed'] = df['Country'].apply(get_latest_confirmed)
df['Population'] = df['Country'].apply(get_population)
df['Density'] = df['Country'].apply(get_density)
df['Cases per Million'] = round(df['Latest Confirmed'] / df['Population'] * 1000000, 2)
country_df = df.set_index('Country')

In [ ]:
df = country_df.nlargest(5,['Latest Confirmed'])
top_ctry = df.index.tolist()
df = confirmed_by_country_df[confirmed_by_country_df['Country'].isin(top_ctry) ]

df['Tag'] = 'Top Countries'

india_cases = country_df.loc['India']['Latest Confirmed']

country_df = country_df.sort_values(['Latest Confirmed'], ascending = False)
df1 = country_df[country_df['Latest Confirmed'] <= india_cases].head(3)
df2 = country_df[country_df['Latest Confirmed'] > india_cases].tail(3)
df1 = df1.append(df2)

ctry_near_india = df1.index.tolist()
df1 = confirmed_by_country_df[confirmed_by_country_df['Country'].isin(ctry_near_india) ]
df1['Tag'] = 'Similar to India'
df = df.append(df1)
scope_ctry = top_ctry + ctry_near_india

In [ ]:
fig = px.line(df, x='Date', y='Confirmed', color='Country', line_group="Tag", labels = 'Tag', hover_name="Country")
fig.update_layout(template='plotly_dark')
fig.show()

**NOTE** <br>
*You can select individual country by double-clicking on the country name on the right*
<br>
*After that you can select more countries by single clicking on the country names on the legend*
<br>
*Once done with comparison, you can double click on legend to show all the countries again*

In the above graph we can see even though China started with a lot of cases how it has flattened the curved by 20th of Feb and since then the new cases are contained very well. <br>
However cases in Italy and US are increasing heavily in a with a steep climb. <br>

# Cases by no. of days

To get a better relative visualization, let me plot no. of cases against no. of days since 100th case for each country. Also it helps to see how each contry have contained the Covid-19 cases after they have witnessed 100th cases. 

To visialize better, we consider only top countries and countries similar to numbers that of India

In [ ]:
df_country_100plus = df_country_100plus[df_country_100plus['Country'].isin(scope_ctry)]

In [ ]:
fig = px.line(df_country_100plus, x='Days', y='Confirmed', color='Country', line_group="Country", hover_name="Country")
fig.update_layout(
    template='plotly_dark',
)

fig.update_xaxes(title_text='No. of days since 100th case')
fig.update_yaxes(title_text='No. of confirmed cases')
fig.show()
py.plot(fig, filename='countrywise-post100cases')

In [ ]:
## How to load the figrure from some accoutn and display in the Jupyter
# fig = py.get_figure('https://plot.ly/~karthikcs/10/')  ## 1st way
# fig = py.get_figure('karthikcs',10) ## 2nd Way to get figure from account
# fig.show()

**NOTE** <br>
*You can select individual country by double-clicking on the country name on the right*
<br>
*After that you can select more countries by single clicking on the country names on the legend*
<br>
*Once done with comparison, you can double click on legend to show all the countries again*

# Cases in India

In [ ]:
fig = plot_aggregate_metrics(country_df.loc['India']['Cases'])
fig.update_layout(title='Aggregate cases in India')
fig.show()
py.plot(fig, filename='covid-india-cases')

We can see that after 4-Mar-2020, the exponential increase of confirmed cases have started. We can observe the same in graph below <br>

On an average Cases are growing day by day with 9% to 12%.

In [ ]:
fig = plot_diff_metrics(country_df.loc['India']['Cases'])
fig.update_layout(title = 'India Metrics: Mortality Rate, Recovery Rate and New Cases rate')
fig.show()

Let us now comapare India with other few countries which are of similar numbers with respect to Covid-19

In [ ]:
df1 = country_df.loc['India']['Cases']
df2 = country_df.loc['US']['Cases']
df3 = country_df.loc['Brazil']['Cases']
df4 = country_df.loc['Japan']['Cases']
df5 = country_df.loc['France']['Cases']

# fig = px.line(df, x='Date', y='Growth Rate')
fig = go.Figure()
fig.add_trace(go.Scatter(x=df4['Date'], 
                             y=df4['Growth Rate'],
                             mode='lines+markers',
                             name='Growth Japan',
                             line=dict(color='Pink', width=2)
             ))

fig.add_trace(go.Scatter(x=df5['Date'], 
                             y=df5['Growth Rate'],
                             mode='lines+markers',
                             name='Growth France',
                             line=dict(color='Grey', width=2)
             ))

fig.add_trace(go.Scatter(x=df3['Date'], 
                             y=df3['Growth Rate'],
                             mode='lines+markers',
                             name='Growth Brazil',
                             line=dict(color='Green', width=2)
             ))

fig.add_trace(go.Scatter(x=df1['Date'], 
                             y=df1['Growth Rate'],
                             mode='lines+markers',
                             name='Growth India',
                             line=dict(color='Blue', width=2)
             ))
fig.add_trace(go.Scatter(x=df2['Date'], 
                             y=df2['Growth Rate'],
                             mode='lines+markers',
                             name='Growth US',
                             line=dict(color='White', width=2)
             ))

fig.update_layout(title = 'Daily Growth Rate India compared with other few countries', template='plotly_dark')
fig.update_yaxes(title_text=' Growth Rate % ')
fig.update_layout(yaxis=dict(tickformat=".2%"))
fig.show()

# Cases vs Population

In [ ]:
df = country_df.reset_index()
df['Cases per Million'] = df['Cases per Million'].apply(lambda x: 0 if( x<0 ) else x)

In [ ]:

# fig = px.scatter( df, x='Population', y='Cases per Million', log_y=True, color="Latest Confirmed",
#                  log_x=True,  hover_name='Country', hover_data=["Latest Confirmed"]
#                 )
# fig.update_layout(template='plotly_dark', title='Covid Cases per Million Population for each country')
# fig.show()

Covid-19 affected cases per million people in each country. We can see that there are many countries who value for cases per million is too high just because their population is too low., e.g.s - **Holy See**: Cases per million is almost 4999. But if you observe the confirmed cases are only 4 (as of 27-Mar-2020). <br>

So let us remove all the countries whos are cases are less than 100. and observe the same information

In [ ]:
df = df[df['Latest Confirmed'] >= 100]
df['Log Confirmed'] = df['Latest Confirmed'].apply(lambda x: np.log(x) )
df.head()

In [ ]:
fig = px.scatter( df, x='Population', y='Latest Confirmed', log_y=True, size = 'Log Confirmed', color="Log Confirmed",
                 log_x=True,  hover_name='Country', hover_data=["Latest Confirmed"]
                )
fig.update_layout(template='plotly_dark', title='Covid Cases vs population for each country')
fig.show()

Note: To visualize better population, Cases per million as well as no. of total cases all are shown in logorithmic scale.
We can see there is sligh correlation with population and confirmed cases. Let us now see how is it influenced by Density of population

In [ ]:
fig = px.scatter( df, x='Density', y='Latest Confirmed', log_y=True, size = 'Log Confirmed', color="Log Confirmed",
                 log_x=True,  hover_name='Country', hover_data=["Latest Confirmed"]
                )
fig.update_layout(template='plotly_dark', title='Covid Cases vs population for each country')
fig.show()

We can see that there is not direct correlation between density of the population and no. of cases

# Death Rate Analysis

Let us consider only contries with more than 500 cases.

In [ ]:
latest_by_country = confirmed_by_country_df.groupby(['Country']).max()
latest_by_country.reset_index(inplace=True)
latest_by_country['Mortality Rate'] = latest_by_country['Deaths'] / latest_by_country['Confirmed']
latest_by_country['Recovery Rate'] = latest_by_country['Recovered'] / latest_by_country['Confirmed']
latest_by_country['Population'] = latest_by_country['Country'].apply(get_population)
latest_by_country.set_index('Country', inplace=True)
# latest_by_country.loc['India']

In [ ]:
death_rate = latest_by_country[latest_by_country['Confirmed'] >= 500 ]
death_rate = death_rate.sort_values(['Mortality Rate','Confirmed'], ascending=False)
death_rate = death_rate.reset_index()
# death_rate.head()

In [ ]:
fig = px.scatter( death_rate, x='Population', y='Confirmed', log_y=True, size = 'Mortality Rate', color = 'Mortality Rate',
                 log_x=True,  hover_name='Country', hover_data=["Recovery Rate"], 
                )
                           
fig.update_layout(template='plotly_dark', title='Country-wise Death Rate Analysis')
fig.show()
# py.plot(fig, filename='covid-country-deathrate')

In the above graph we can see the death rate is denoted by the size and color for the countries whose Covid cases are more than 500. We can observe the India's death rate 2.75%. Italy's mortality case is 12.3% which is highest, followed by Algeria with 11.5% and UK which is 10.2%. Let us see top 5 countries with death rates

In [ ]:
death_rate.head()[['Country','Mortality Rate']]

In [ ]:
max_xvalue = confirmed_by_country_df.Confirmed.max()
max_xvalue = np.log10(max_xvalue) + 1

max_yvalue = confirmed_by_country_df.Deaths.max()
max_yvalue = np.log10(max_yvalue) + 1


fig = px.scatter( confirmed_by_country_df, x='Confirmed', y='Deaths', log_y=True,  size = 'Deaths', color = 'Deaths',
                 log_x=True,  hover_name='Country', animation_frame="Date", animation_group="Country", text='Country',
                )
                           
fig.update_layout(template='plotly_dark', title='Country-wise Death Rate Analysis', )
fig.update_xaxes(range=[0, max_xvalue])
fig.update_yaxes(range=[0, max_yvalue])
fig.show()
# py.plot(fig, filename='covid-deathrate-animation')

In the next graph let us see the heat map of death rate on the world map

In [ ]:
fig = px.choropleth(death_rate, locations="Country",locationmode='country names', color="Mortality Rate",
                     hover_name="Country", color_continuous_scale = 'reds',
                     projection="natural earth",title='Death Rate each country')
py.plot(fig, filename='covid-deathrate-heatmap')
fig.show()

Following are the countries with top recovery rates

In [ ]:
death_rate = death_rate.sort_values('Recovery Rate', ascending = False)

death_rate.head()

We can see New Zealand has highest recovery rate with **97.2%**

# Finding tipping points for countries
No. of daily cases vs Total no. of cases

In [ ]:
def plot_tipping_curve(df, color, fig=None, ):
    if fig is None:
        fig = go.Figure()
        fig.update_layout(template='plotly_dark', xaxis_title="Confirmed cases", yaxis_title="Daily New Cases", title="Covid Cases: Tipping point analysis",)
        fig.update_layout(xaxis_type="log")
        fig.update_layout(yaxis_type="log")

    df1 = df[df['Confirmed Change'] > 0]
    country = df1.iloc[0]['Country']
    
    fig.add_trace(go.Scatter(x=df1['Confirmed'], 
                             y=df1['Confirmed Change'],
                             mode='lines+markers',
                             name= country,
                             line=dict(color=color, width=1)))

    return fig

Trying to understand data at that poing when the new cases go down. Let us calculate the ratio New Cases / Existing cases (Let us use log10 while dividing). Try to get the Moving Average graph of that data  to analyse further.

In [ ]:
def plot_ratio_sma(df, color, fig, row=1, col=1):
    df['ratio'] = np.log10(df['Confirmed Change']) / np.log10(df['Confirmed'])
    df['sma'] = df.ratio.rolling(5).mean()
    fig.append_trace(go.Scatter(x=df['Date'], 
                             y=df['ratio'],
                             mode='lines',
                             name= 'Ratio of New Cases / Cases',
                             line=dict(color=color, width=1)), row=row, col=col)

    fig.add_trace(go.Scatter(x=df['Date'], 
                                 y=df['sma'],
                                 mode='lines+markers',
                                 name= 'Moving avg of ratio',
                                 line=dict(color=color, width=1)), row=row, col=col )
    return fig
    

In [ ]:
fig = make_subplots(rows=3, cols=2)
fig.update_layout(template='plotly_dark', title="Covid Cases: New cases vs Total Cases Ratio and Moving Average",showlegend=False, )


df = country_df.loc['India']['Cases']
fig = plot_ratio_sma(df, 'blue', fig, 1,1)
fig['layout']['xaxis1'].update(showticklabels=False)
fig['layout']['yaxis1'].update(title='India')

df = country_df.loc['US']['Cases']
fig = plot_ratio_sma(df, 'red', fig, 1,2)
fig['layout']['xaxis2'].update(showticklabels=False)
fig['layout']['yaxis2'].update(title='USA')

df = country_df.loc['Mainland China']['Cases']
fig = plot_ratio_sma(df, 'white', fig, 2,1)
fig['layout']['xaxis3'].update(showticklabels=False)
fig['layout']['yaxis3'].update(title='China')

df = country_df.loc['Italy']['Cases']
fig = plot_ratio_sma(df, 'grey', fig, 2,2)
fig['layout']['xaxis4'].update(showticklabels=False)
fig['layout']['yaxis4'].update(title='Italy')


df = country_df.loc['South Korea']['Cases']
fig = plot_ratio_sma(df, 'green', fig, 3,1)
fig['layout']['xaxis5'].update(showticklabels=False)
fig['layout']['yaxis5'].update(title='S Korea')

df = country_df.loc['Philippines']['Cases']
fig = plot_ratio_sma(df, 'magenta', fig, 3,2)
fig['layout']['xaxis6'].update(showticklabels=False)
fig['layout']['yaxis6'].update(title='Philippines')


fig.show()



``` 9-Apr-2020```

We can observe before cases get stabilized, the ration between new cases vs existing cases reduce constantly. Example: S Korea, Italy. But in case of India and USA we can see the cases are still growing and it will continue to grow


In [ ]:
df = country_df.loc['India']['Cases']
fig = plot_tipping_curve(df, color='blue')

df = country_df.loc['Japan']['Cases']
fig = plot_tipping_curve(df,color='green', fig=fig)

df = country_df.loc['Mainland China']['Cases']
fig = plot_tipping_curve(df,color='yellow', fig=fig)

df = country_df.loc['US']['Cases']
fig = plot_tipping_curve(df,color='pink', fig=fig)

df = country_df.loc['South Korea']['Cases']
fig = plot_tipping_curve(df,color='orange', fig=fig)

df = country_df.loc['Italy']['Cases']
fig = plot_tipping_curve(df,color='red', fig=fig)

df = country_df.loc['Spain']['Cases']
fig = plot_tipping_curve(df,color='brown', fig=fig)

fig.update_layout( annotations=[
                    dict(
                        x=np.log10(16600),
                        y=np.log10(4736),
                        xref="x",
                        yref="y",
                        text="China Tipping",
                        showarrow=True,
                        arrowhead=6,
                        ax=0,
                        ay=-40),
    
                    dict(
                        x=np.log10(5186),
                        y=np.log10(851),
                        xref="x",
                        yref="y",
                        text="S. Korea Tipping",
                        showarrow=True,
                        arrowhead=6,
                        ax=0,
                        ay=-40),
                    dict(
                        x=np.log10(53578),
                        y=np.log10(6557),
                        xref="x",
                        yref="y",
                        text="Italy Tipping",
                        showarrow=True,
                        arrowhead=6,
                        ax=0,
                        ay=-40),
                    ])
fig.show()
py.plot(fig, filename='covid-cases-tipping-downwards')

We can observe that South Korea and China have achieved tipping points. China have reached it once they hit around 60K confirmed cases. South Korea have reached tipping point when they were 7K cases. The common pattern what we can see is, at certain level, no. of new cases will not increase compared to total no. of cases. In China we can observe that no. of daily new cases started to decrease once they hit around 16K and slowly decreased and attained the tipping point. Same in case of S. Korea. 

```9-Apr-2020```

We can see that Italy is now getting same amount of cases even though total cases are increasing. That is  an indication that it has attained the tipping point

By seeing above graphs, we can observe that before any country flattens the curve (a.k.a tip down), for 

# Modelling data and Projection

Using Polynomial regression let us try to get the right model for different contries and project for next few days

In [ ]:
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split

Projecting US Data - Total cases

In [ ]:
df = country_df.loc['US']['Cases']
df = df.fillna(0)
X = df[['Days']]
y = df['Confirmed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
fig=go.Figure()
fig.update_layout(template='plotly_dark', title="US new cases", yaxis_title="New Cases", xaxis_title="No. of days",)
fig.add_trace(go.Scatter(name='US New cases', x=df['Date'], y=y, mode='lines'))
fig.show()

Find which polynomial degreee suits the curve

In [ ]:
for i in range(1,10):
    poly = PolynomialFeatures(degree = i)
    regr = linear_model.LinearRegression()
    X_ = poly.fit_transform(X_train)
    regr.fit(X_, y_train)
    X_ = poly.fit_transform(X_test)
    results = regr.predict(X_)
    score1 = metrics.mean_squared_error(y_test,results )
    score2 = metrics.r2_score(y_test,results )
    print('Model with Polynominal Degree', i, 'MSE: ',score1, '  R2 Score: ', score2)

Looking at R2 Scores, we have best scrore at Degree 7. However, that will be overfittign the curve, which we should avoid. To get a simple model which generalizes the data points, we can choose Degree 3

In [ ]:
poly = PolynomialFeatures(degree = 6)
regr = linear_model.LinearRegression()
X_ = poly.fit_transform(X)
regr.fit(X_, y)
results = regr.predict(X_)

no_of_days = df['Days'].max() + 60
X_plot=np.linspace(0,no_of_days,200).reshape(-1,1)
X_plot_poly=poly.fit_transform(X_plot)
fig=go.Figure()
fig.update_layout(template='plotly_dark', title="Projection of US cases for next 30 days", yaxis_title="Cases", xaxis_title="No. of days",)
fig.add_trace(go.Scatter(name='Projection', x=X_plot_poly[:,1], y=regr.predict(X_plot_poly), mode='markers'))
fig.add_trace(go.Scatter(name='Current cases', x=df['Days'], y=y, mode='lines'))
fig.update_yaxes(range=[0,1000000])
fig.show()

US - Analysis of Daily new cases

In [ ]:
df = country_df.loc['US']['Cases']
df = df.fillna(0)
X = df[['Days']]
y = df['Confirmed Change']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
fig=go.Figure()
fig.update_layout(template='plotly_dark', title="US Daily New Cases", yaxis_title="New Cases", xaxis_title="Date",)
fig.add_trace(go.Scatter(name='US New cases', x=df['Date'], y=y, mode='lines'))
fig.show()

Find which polynomial degreee suits the curve

In [ ]:
for i in range(1,10):
    poly = PolynomialFeatures(degree = i)
    regr = linear_model.LinearRegression()
    X_ = poly.fit_transform(X_train)
    regr.fit(X_, y_train)
    X_ = poly.fit_transform(X_test)
    results = regr.predict(X_)
    score1 = metrics.mean_squared_error(y_test,results )
    score2 = metrics.r2_score(y_test,results )
    print('Model with Polynominal Degree', i, 'MSE: ',score1, '  R2 Score: ', score2)

Looking at R2 Scores, we have best scrore at Degree 7. However, that will be overfittign the curve, which we should avoid. To get a simple model which generalizes the data points, we can choose Degree 3

In [ ]:
poly = PolynomialFeatures(degree = 3)
regr = linear_model.LinearRegression()
X_ = poly.fit_transform(X)
regr.fit(X_, y)
results = regr.predict(X_)

no_of_days = df['Days'].max() + 30
X_plot=np.linspace(0,no_of_days,200).reshape(-1,1)
X_plot_poly=poly.fit_transform(X_plot)
fig=go.Figure()
fig.update_layout(template='plotly_dark', title="US: Projection of new cases for next 30 days", yaxis_title="Cases", xaxis_title="No. of days since 100th case",)
fig.add_trace(go.Scatter(name='Projection', x=X_plot_poly[:,1], y=regr.predict(X_plot_poly), mode='markers'))
fig.add_trace(go.Scatter(name='Current cases', x=df['Days'], y=df['Confirmed Change'], mode='lines'))
fig.update_yaxes(range=[0,40000])
fig.show()

In [ ]:
df = country_df.loc['India']['Cases']
df = df.fillna(0)
X = df[['Days']]
y = df['Confirmed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
for i in range(1,10):
    poly = PolynomialFeatures(degree = i)
    regr = linear_model.LinearRegression()
    X_ = poly.fit_transform(X_train)
    regr.fit(X_, y_train)
    X_ = poly.fit_transform(X_test)
    results = regr.predict(X_)
    score1 = metrics.mean_squared_error(y_test,results )
    score2 = metrics.r2_score(y_test,results )
    print('Model with Polynominal Degree', i, 'MSE: ',score1, '  R2 Score: ', score2)

Looking at R2 Scores, we have best scrore at Degree 5. However, that will be overfittign the curve, which we should avoid. To get a simple model which generalizes the data points, we can choose Degree 3 to 4.

In [ ]:
poly1 = PolynomialFeatures(degree = 4)
regr1 = linear_model.LinearRegression()
X1_ = poly1.fit_transform(X)
regr1.fit(X1_, y)
# results = regr1.predict(X1_)

poly2 = PolynomialFeatures(degree = 5)
regr2 = linear_model.LinearRegression()
X2_ = poly2.fit_transform(X)
regr2.fit(X2_, y)


no_of_days = df['Days'].max() + 60
X1_plot=np.linspace(0,no_of_days,100).reshape(-1,1)
X1_plot_poly=poly1.fit_transform(X1_plot)

X2_plot=np.linspace(0,no_of_days,100).reshape(-1,1)
X2_plot_poly=poly2.fit_transform(X2_plot)


fig=go.Figure()
fig.update_layout(template='plotly_dark', title="Projection of India Total cases for next 30 days", yaxis_title="Cases", xaxis_title="No. of days")
fig.add_trace(go.Scatter(name='Projection Degree 3', x=X1_plot_poly[:,1], y=regr1.predict(X1_plot_poly), mode='markers'))
fig.add_trace(go.Scatter(name='Projection Degree 4', x=X2_plot_poly[:,1], y=regr2.predict(X2_plot_poly), mode='markers'))
fig.add_trace(go.Scatter(name='Current cases', x=df['Days'], y=y, mode='lines'))
fig.update_yaxes(range=[0,6000000])
fig.show()
py.plot(fig, filename='covid-india-projection-deg2')

```9-Apr-2020```

We can see that using Poly degree 3, India will have about 86,000 cases in next 60 days. If you observe in the projection. 
With Poly degree 4, India will have about 173,000 cases in next 60 days. 



India: Projecting Daily new Cases. This will allow us to know when the curve gets faltten

In [ ]:
df = country_df.loc['India']['Cases']
df = df.fillna(0)
X = df[['Days']]
y = df['Confirmed Change']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
for i in range(1,10):
    poly = PolynomialFeatures(degree = i)
    regr = linear_model.LinearRegression()
    X_ = poly.fit_transform(X_train)
    regr.fit(X_, y_train)
    X_ = poly.fit_transform(X_test)
    results = regr.predict(X_)
    score1 = metrics.mean_squared_error(y_test,results )
    score2 = metrics.r2_score(y_test,results )
    print('Model with Polynominal Degree', i, 'MSE: ',score1, '  R2 Score: ', score2)

Looking at R2 Scores, we have best scrore at Degree 4. However, that will be overfittign the curve, which we should avoid. To get a simple model which generalizes the data points, we can choose Degree 3

In [ ]:
poly1 = PolynomialFeatures(degree = 2)
regr1 = linear_model.LinearRegression()
X1_ = poly1.fit_transform(df[['Days']])
regr1.fit(X1_, df['Confirmed Change'])
# results = regr1.predict(X1_)

poly2 = PolynomialFeatures(degree = 4)
regr2 = linear_model.LinearRegression()
X2_ = poly2.fit_transform(df[['Days']])
regr2.fit(X2_, df['Confirmed Change'])


no_of_days = df['Days'].max() + 60
X1_plot=np.linspace(0,no_of_days,100).reshape(-1,1)
X1_plot_poly=poly1.fit_transform(X1_plot)

X2_plot=np.linspace(0,no_of_days,100).reshape(-1,1)
X2_plot_poly=poly2.fit_transform(X2_plot)


fig=go.Figure()
fig.update_layout(template='plotly_dark', title="Projection of India Daily cases for next 30 days", yaxis_title="Cases", xaxis_title="No. of days")
fig.add_trace(go.Scatter(name='Projection Degree 3', x=X1_plot_poly[:,1], y=regr1.predict(X1_plot_poly), mode='markers'))
fig.add_trace(go.Scatter(name='Projection Degree 4', x=X2_plot_poly[:,1], y=regr2.predict(X2_plot_poly), mode='markers'))
fig.add_trace(go.Scatter(name='Current cases', x=df['Days'], y=df['Confirmed Change'], mode='lines'))
fig.update_yaxes(range=[0,200000])
fig.show()
py.plot(fig, filename='covid-india-projection-deg2')

`17-Apr-2020`

As per above graph we can see the number of daily cases would start to decrease from Day 50 (from 100th Case), which is 8-May-2020. Then, it keeps reducing and by 30-Jun-2020, the Corona should be under control

`12-Aug-2020`
All my predictions are gone for toss :) hahaa